In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))


from utils import *
import utils as ut

In [2]:
whos

Variable                 Type        Data/Info
----------------------------------------------
ADASYN                   ABCMeta     <class 'imblearn.over_sampling._adasyn.ADASYN'>
BaseEstimator            type        <class 'sklearn.base.BaseEstimator'>
BorderlineSMOTE          ABCMeta     <class 'imblearn.over_sam<...>.filter.BorderlineSMOTE'>
DecisionTreeClassifier   ABCMeta     <class 'sklearn.tree._cla<...>.DecisionTreeClassifier'>
GridSearchCV             ABCMeta     <class 'sklearn.model_sel<...>on._search.GridSearchCV'>
KFold                    ABCMeta     <class 'sklearn.model_selection._split.KFold'>
KMeansSMOTE              ABCMeta     <class 'imblearn.over_sam<...>ote.cluster.KMeansSMOTE'>
LGBMClassifier           type        <class 'lightgbm.sklearn.LGBMClassifier'>
LinearSVC                type        <class 'sklearn.svm._classes.LinearSVC'>
LogisticRegression       type        <class 'sklearn.linear_mo<...>stic.LogisticRegression'>
MinMaxScaler             type        <cla

# < 전처리 경우의 수 > 


### 이상 index  삭제 16가지

- 원핫인코딩 적용 8가지
    - NULL값 적용 4가지 (스케일링 - MinMax, Standard, Robuster, 없음)
    - NULL값 미적용 4가지 (스케일링 - MinMax, Standard, Robuster, 없음)
        
- 원핫인코딩 미적용 8가지
    - NULL값 적용 4가지 (스케일링 - MinMax, Standard, Robuster, 없음)
    - NULL값 미적용 4가지 (스케일링 - MinMax, Standard, Robuster, 없음)
    
----------------------------------------------------------------------------------------------

## 총 16가지

# < 순서 >

### 1. 원핫인코딩 쓰지 않는 경우 - 8가지 
### 2. 원핫인코딩 쓰는 경우 - 8가지 

## 총 16가지



## 데이터 불러오기

## 1. 이상 index 삭제, 원핫인코딩 쓰지 않는 경우 - 8가지 

### 이상치 인덱스(대여시간, 사고시간, 대여기간 각각 맞지 않는 경우) drop

In [3]:
df = pd.read_csv("../../insurance_fraud_detect_data.csv")

# 41개 삭제
strange = [43, 1504, 1762, 11706, 12031, 12665, 480, 4515, 4651, 8596, 9904, 10635, 11104, 11120, 11994, 12956,
      12972, 14533, 14792, 15242, 209, 268, 12132, 8477, 12705, 8514, 532, 4158, 5401, 13077, 4595, 6839,
      11710, 12377, 15080, 1115, 5398, 5790, 9523, 9528, 12050]
socar = df.drop(strange)

socar.columns = ["c_"+str(i) for i in range(1,26)]
socar

,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,...,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25
0,0,2,1,3,0,2,0,100,2,0,...,0,1,0,0,1,0,0,0,-1,0
1,0,1,0,4,0,4,0,0,3,1,...,0,1,1,1,2,1,1,1,0,1
2,0,1,0,4,1,4,0,100,3,1,...,12925040,3,1,0,2,1,1,1,0,1
3,0,3,1,3,0,4,0,0,2,0,...,0,3,1,0,1,0,0,0,-1,0
4,0,1,0,1,0,1,0,100,1,0,...,1209410,1,0,0,2,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,2,0,2,0,2,0,100,2,0,...,0,1,1,0,1,0,0,0,-1,0
15996,0,2,0,2,1,4,0,100,1,1,...,0,2,1,0,1,0,0,0,-1,0
15997,0,2,1,2,0,4,0,100,1,0,...,0,0,0,0,1,0,0,0,-1,1
15998,0,2,0,2,0,2,0,100,2,0,...,0,4,1,0,1,0,0,0,-1,0


#### < 1. 첫번째 방법 (NULL 유지 , SMOTE, 스케일링 없음) >

- (1) NULL 유지, (2) 데이터 분리

In [4]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar[socar["c_25"] == 0]
test_set = socar[socar["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (3) SMOTE

In [5]:
#  X : test column = 0 , y : test column = 1

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)  

In [6]:
X.shape, y.shape

((12838, 23), (12838,))

In [7]:
X_train_over.shape, y_train_over.shape

((25608, 23), (25608,))

In [8]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (4) 스케일링 없음

#### < 2. 두번째 방법 (NULL 유지 , SMOTE, 스케일링 (MinMax)) >

- (1) NULL 유지, (2) 데이터 분리

In [9]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar[socar["c_25"] == 0]
test_set = socar[socar["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (3) SMOTE

In [10]:
#  X : test column = 0 , y : test column = 1

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)  

In [11]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (4) 스케일링 (MinMax)

In [12]:
from sklearn.preprocessing import MinMaxScaler

MMS = MinMaxScaler()

MMS.fit(X)

X_mms = MMS.transform(X)

In [13]:
X_mms_pd2 = pd.DataFrame(X_mms, columns=X.columns)
X_mms_pd2

,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,...,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24
0,0.25,1.0,0.50,0.0,0.333333,0.0,1.0,0.5,0.0,0.0,...,0.013409,0.000000,0.2,0.0,0.0,0.027027,0.000000,0.0,0.0,0.0
1,0.50,1.0,0.50,0.0,1.000000,0.0,0.0,0.5,0.0,0.0,...,0.138052,0.000000,0.6,1.0,0.0,0.027027,0.000000,0.0,0.0,0.0
2,0.00,0.0,0.00,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,...,0.053947,0.012769,0.2,0.0,0.0,0.054054,0.000000,0.0,0.0,0.0
3,0.00,1.0,0.50,0.0,1.000000,0.0,1.0,1.0,0.0,0.0,...,0.000000,0.012852,0.0,0.0,0.0,0.027027,0.333333,1.0,1.0,0.5
4,0.25,0.0,0.75,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.2,0.0,0.0,0.027027,0.333333,0.5,1.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,0.25,0.0,0.50,1.0,1.000000,0.0,1.0,0.0,0.0,0.0,...,0.068696,0.000000,0.0,0.0,0.0,0.027027,0.000000,0.0,0.0,0.0
12834,0.25,0.0,0.25,0.0,0.333333,0.0,1.0,0.5,0.0,0.0,...,0.000000,0.000000,0.2,1.0,0.0,0.027027,0.000000,0.0,0.0,0.0
12835,0.25,0.0,0.25,1.0,1.000000,0.0,1.0,0.0,1.0,0.0,...,0.063666,0.000000,0.4,1.0,0.0,0.027027,0.000000,0.0,0.0,0.0
12836,0.25,0.0,0.25,0.0,0.333333,0.0,1.0,0.5,0.0,0.0,...,0.000000,0.000000,0.8,1.0,0.0,0.027027,0.000000,0.0,0.0,0.0


#### < 3. 세번째 방법 (NULL 유지 , SMOTE, 스케일링 (Standard)) >

- (1) NULL 유지, (2) 데이터 분리

In [14]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar[socar["c_25"] == 0]
test_set = socar[socar["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (3) SMOTE

In [15]:
#  X : test column = 0 , y : test column = 1

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)  

In [16]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (4) 스케일링 (Standard)

In [17]:
from sklearn.preprocessing import StandardScaler

SS = StandardScaler()

SS.fit(X)

X_ss = SS.transform(X)

In [18]:
X_ss_pd3 = pd.DataFrame(X_ss, columns=X.columns)
X_ss_pd3

,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,...,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24
0,0.394112,2.371572,0.445725,-0.286048,-0.456268,-0.078687,0.368445,0.330754,-0.389752,-0.27735,...,-0.296987,-0.273447,-0.221541,-0.696804,-0.45426,-0.381907,-0.549077,-0.533844,-0.574652,-0.484791
1,1.703194,2.371572,0.445725,-0.286048,1.261997,-0.078687,-3.004413,0.330754,-0.389752,-0.27735,...,2.053374,-0.273447,1.279722,1.435124,-0.45426,-0.381907,-0.549077,-0.533844,-0.574652,-0.484791
2,-0.914971,-0.421661,-1.260616,-0.286048,-1.315400,-0.078687,0.368445,-1.049681,-0.389752,-0.27735,...,0.467425,0.120768,-0.221541,-0.696804,-0.45426,0.671804,-0.549077,-0.533844,-0.574652,-0.484791
3,-0.914971,2.371572,0.445725,-0.286048,1.261997,-0.078687,0.368445,1.711188,-0.389752,-0.27735,...,-0.549843,0.123334,-0.972173,-0.696804,-0.45426,-0.381907,1.608576,2.912721,1.740184,2.648493
4,0.394112,-0.421661,1.298895,-0.286048,1.261997,-0.078687,-3.004413,-1.049681,-0.389752,-0.27735,...,-0.549843,-0.273447,-0.221541,-0.696804,-0.45426,-0.381907,1.608576,1.189438,1.740184,2.648493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,0.394112,-0.421661,0.445725,3.495915,1.261997,-0.078687,0.368445,-1.049681,-0.389752,-0.27735,...,0.745531,-0.273447,-0.972173,-0.696804,-0.45426,-0.381907,-0.549077,-0.533844,-0.574652,-0.484791
12834,0.394112,-0.421661,-0.407446,-0.286048,-0.456268,-0.078687,0.368445,0.330754,-0.389752,-0.27735,...,-0.549843,-0.273447,-0.221541,1.435124,-0.45426,-0.381907,-0.549077,-0.533844,-0.574652,-0.484791
12835,0.394112,-0.421661,-0.407446,3.495915,1.261997,-0.078687,0.368445,-1.049681,2.565734,-0.27735,...,0.650695,-0.273447,0.529091,1.435124,-0.45426,-0.381907,-0.549077,-0.533844,-0.574652,-0.484791
12836,0.394112,-0.421661,-0.407446,-0.286048,-0.456268,-0.078687,0.368445,0.330754,-0.389752,-0.27735,...,-0.549843,-0.273447,2.030354,1.435124,-0.45426,-0.381907,-0.549077,-0.533844,-0.574652,-0.484791


#### < 4. 네번째 방법 (NULL 유지 , SMOTE, 스케일링 (Robust)) >

- (1) NULL 유지, (2) 데이터 분리

In [19]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar[socar["c_25"] == 0]
test_set = socar[socar["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (3) SMOTE

In [20]:
#  X : test column = 0 , y : test column = 1

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)  

In [21]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (4) 스케일링 (Standard)

In [22]:
from sklearn.preprocessing import RobustScaler

RS = RobustScaler()

RS.fit(X)

X_rs = RS.transform(X)

In [23]:
X_rs_pd4 = pd.DataFrame(X_rs, columns=X.columns)
X_rs_pd4

,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,...,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.161577,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0.0,1.0,0.0,-100.0,0.0,0.0,0.0,...,3.179712,0.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.0,0.0,-1.0,0.0,-0.5,0.0,0.0,-1.0,0.0,0.0,...,0.925116,2.175491,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.521038,2.189648,-0.5,0.0,0.0,0.0,1.0,2.0,1.0,3.0
4,0.0,0.0,2.0,0.0,1.0,0.0,-100.0,-1.0,0.0,0.0,...,-0.521038,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,0.0,0.0,1.0,1.0,1.0,0.0,0.0,-1.0,0.0,0.0,...,1.320474,0.000000,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.521038,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12835,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-1.0,1.0,0.0,...,1.185654,0.000000,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.521038,0.000000,1.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0


#### < 5. 다섯번째 방법 (NULL 삭제 , SMOTE, 스케일링 없음) >

- (1) NULL값 삭제 

- null값이 들어있는 컬럼
    - c_14 : -1 - 36개
    - c_15 : 0 - 6006개
    - c_16 : 0 - 10424개
    - c_17 : 5(확인불가) - 288개
    - c_21 : 0 - 11384개
    - c_22 : 0 - 11325개
    - c_23 :  0 - 11328개
    - c_24 :  -1 - 11358개

In [24]:
socar_strange_index_delete = socar.drop(['c_14', 'c_15', 'c_16', 'c_17', 'c_21', 'c_22', 'c_23', 'c_24'], axis=1)
socar_strange_index_delete

,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_18,c_19,c_20,c_25
0,0,2,1,3,0,2,0,100,2,0,0,4,2,0,0,1,0
1,0,1,0,4,0,4,0,0,3,1,0,1,4,1,1,2,1
2,0,1,0,4,1,4,0,100,3,1,0,1,4,1,0,2,1
3,0,3,1,3,0,4,0,0,2,0,0,3,3,1,0,1,0
4,0,1,0,1,0,1,0,100,1,0,0,1,1,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,2,0,2,0,2,0,100,2,0,0,1,1,1,0,1,0
15996,0,2,0,2,1,4,0,100,1,1,0,5,2,1,0,1,0
15997,0,2,1,2,0,4,0,100,1,0,0,1,3,0,0,1,1
15998,0,2,0,2,0,2,0,100,2,0,0,6,1,1,0,1,0


- (2) 데이터 분리

In [25]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar_strange_index_delete[socar_strange_index_delete["c_25"] == 0]
test_set = socar_strange_index_delete[socar_strange_index_delete["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (3) SMOTE

In [26]:
#  X : test column = 0 , y : test column = 1

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)  

In [27]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (4) 스케일링 없음

#### < 6. 여섯번째 방법 (NULL 삭제 , SMOTE, 스케일링 (MinMax)) >

- (1) NULL 삭제, (2) 데이터 분리

In [28]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar_strange_index_delete[socar_strange_index_delete["c_25"] == 0]
test_set = socar_strange_index_delete[socar_strange_index_delete["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (3) SMOTE

In [29]:
#  X : test column = 0 , y : test column = 1

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)  

In [30]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (4) 스케일링 (MinMax)

In [31]:
from sklearn.preprocessing import MinMaxScaler

MMS = MinMaxScaler()

MMS.fit(X)

X_mms = MMS.transform(X)

In [32]:
X_mms_pd6 = pd.DataFrame(X_mms, columns=X.columns)
X_mms_pd6


,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_18,c_19,c_20
0,0.25,1.0,0.50,0.0,0.333333,0.0,1.0,0.5,0.0,0.0,0.6,0.25,0.0,0.0,0.027027
1,0.50,1.0,0.50,0.0,1.000000,0.0,0.0,0.5,0.0,0.0,0.4,0.50,1.0,0.0,0.027027
2,0.00,0.0,0.00,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.054054
3,0.00,1.0,0.50,0.0,1.000000,0.0,1.0,1.0,0.0,0.0,1.0,0.50,0.0,0.0,0.027027
4,0.25,0.0,0.75,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.6,0.25,0.0,0.0,0.027027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,0.25,0.0,0.50,1.0,1.000000,0.0,1.0,0.0,0.0,0.0,0.6,0.00,0.0,0.0,0.027027
12834,0.25,0.0,0.25,0.0,0.333333,0.0,1.0,0.5,0.0,0.0,0.0,0.00,1.0,0.0,0.027027
12835,0.25,0.0,0.25,1.0,1.000000,0.0,1.0,0.0,1.0,0.0,0.8,0.25,1.0,0.0,0.027027
12836,0.25,0.0,0.25,0.0,0.333333,0.0,1.0,0.5,0.0,0.0,1.0,0.00,1.0,0.0,0.027027


#### < 7. 일곱번째 방법 (NULL 삭제 , SMOTE, 스케일링  (Standard)) >

- (1) NULL 삭제, (2) 데이터 분리

In [33]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar_strange_index_delete[socar_strange_index_delete["c_25"] == 0]
test_set = socar_strange_index_delete[socar_strange_index_delete["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (3) SMOTE

In [34]:
#  X : test column = 0 , y : test column = 1

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)  

In [35]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (4) 스케일링 (Standard)

In [36]:
from sklearn.preprocessing import StandardScaler

SS = StandardScaler()

SS.fit(X)

X_ss = SS.transform(X)

In [37]:
X_ss_pd7 = pd.DataFrame(X_ss, columns=X.columns)
X_ss_pd7

,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_18,c_19,c_20
0,0.394112,2.371572,0.445725,-0.286048,-0.456268,-0.078687,0.368445,0.330754,-0.389752,-0.27735,0.155173,-0.217666,-0.696804,-0.45426,-0.381907
1,1.703194,2.371572,0.445725,-0.286048,1.261997,-0.078687,-3.004413,0.330754,-0.389752,-0.27735,-0.425957,0.645867,1.435124,-0.45426,-0.381907
2,-0.914971,-0.421661,-1.260616,-0.286048,-1.315400,-0.078687,0.368445,-1.049681,-0.389752,-0.27735,-1.588216,-1.081198,-0.696804,-0.45426,0.671804
3,-0.914971,2.371572,0.445725,-0.286048,1.261997,-0.078687,0.368445,1.711188,-0.389752,-0.27735,1.317433,0.645867,-0.696804,-0.45426,-0.381907
4,0.394112,-0.421661,1.298895,-0.286048,1.261997,-0.078687,-3.004413,-1.049681,-0.389752,-0.27735,0.155173,-0.217666,-0.696804,-0.45426,-0.381907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,0.394112,-0.421661,0.445725,3.495915,1.261997,-0.078687,0.368445,-1.049681,-0.389752,-0.27735,0.155173,-1.081198,-0.696804,-0.45426,-0.381907
12834,0.394112,-0.421661,-0.407446,-0.286048,-0.456268,-0.078687,0.368445,0.330754,-0.389752,-0.27735,-1.588216,-1.081198,1.435124,-0.45426,-0.381907
12835,0.394112,-0.421661,-0.407446,3.495915,1.261997,-0.078687,0.368445,-1.049681,2.565734,-0.27735,0.736303,-0.217666,1.435124,-0.45426,-0.381907
12836,0.394112,-0.421661,-0.407446,-0.286048,-0.456268,-0.078687,0.368445,0.330754,-0.389752,-0.27735,1.317433,-1.081198,1.435124,-0.45426,-0.381907


#### < 8. 여덟번째 방법 (NULL 삭제 , SMOTE, 스케일링 (Robust))) >

- (1) NULL 삭제, (2) 데이터 분리

In [38]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar_strange_index_delete[socar_strange_index_delete["c_25"] == 0]
test_set = socar_strange_index_delete[socar_strange_index_delete["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (3) SMOTE

In [39]:
#  X : test column = 0 , y : test column = 1

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)  

In [40]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (4) 스케일링 (Robust)

In [41]:
from sklearn.preprocessing import RobustScaler

RS = RobustScaler()

RS.fit(X)

X_rs = RS.transform(X)

In [42]:
X_rs_pd8 = pd.DataFrame(X_rs, columns=X.columns)
X_rs_pd8

,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_18,c_19,c_20
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0.0,1.0,0.0,-100.0,0.0,0.0,0.0,-0.333333,0.5,1.0,0.0,0.0
2,-1.0,0.0,-1.0,0.0,-0.5,0.0,0.0,-1.0,0.0,0.0,-1.000000,-0.5,0.0,0.0,1.0
3,-1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.666667,0.5,0.0,0.0,0.0
4,0.0,0.0,2.0,0.0,1.0,0.0,-100.0,-1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,0.0,0.0,1.0,1.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.000000,-0.5,0.0,0.0,0.0
12834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.000000,-0.5,1.0,0.0,0.0
12835,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-1.0,1.0,0.0,0.333333,0.0,1.0,0.0,0.0
12836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,-0.5,1.0,0.0,0.0


## 2. 이상 index 삭제, 원핫인코딩 쓰는 경우 - 8가지 

In [43]:
socar

,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,...,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25
0,0,2,1,3,0,2,0,100,2,0,...,0,1,0,0,1,0,0,0,-1,0
1,0,1,0,4,0,4,0,0,3,1,...,0,1,1,1,2,1,1,1,0,1
2,0,1,0,4,1,4,0,100,3,1,...,12925040,3,1,0,2,1,1,1,0,1
3,0,3,1,3,0,4,0,0,2,0,...,0,3,1,0,1,0,0,0,-1,0
4,0,1,0,1,0,1,0,100,1,0,...,1209410,1,0,0,2,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,2,0,2,0,2,0,100,2,0,...,0,1,1,0,1,0,0,0,-1,0
15996,0,2,0,2,1,4,0,100,1,1,...,0,2,1,0,1,0,0,0,-1,0
15997,0,2,1,2,0,4,0,100,1,0,...,0,0,0,0,1,0,0,0,-1,1
15998,0,2,0,2,0,2,0,100,2,0,...,0,4,1,0,1,0,0,0,-1,0


In [44]:
socar_strange_index

,fraud_YN,car_model,sharing_type,age_group,has_previous_accident,cumulative_use_count,b2b,accident_ratio,pf_type,socarpass,...,insure_cost,accident_location,car_part1,car_part2,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
0,0,2,1,3,0,2,0,100,2,0,...,0,1,0,0,1,0,0,0,-1,0
1,0,1,0,4,0,4,0,0,3,1,...,0,1,1,1,2,1,1,1,0,1
2,0,1,0,4,1,4,0,100,3,1,...,12925040,3,1,0,2,1,1,1,0,1
3,0,3,1,3,0,4,0,0,2,0,...,0,3,1,0,1,0,0,0,-1,0
4,0,1,0,1,0,1,0,100,1,0,...,1209410,1,0,0,2,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,2,0,2,0,2,0,100,2,0,...,0,1,1,0,1,0,0,0,-1,0
15996,0,2,0,2,1,4,0,100,1,1,...,0,2,1,0,1,0,0,0,-1,0
15997,0,2,1,2,0,4,0,100,1,0,...,0,0,0,0,1,0,0,0,-1,1
15998,0,2,0,2,0,2,0,100,2,0,...,0,4,1,0,1,0,0,0,-1,0


### ONE-HOT-ENCODER

In [47]:
cat_attribs = socar.columns.drop(['c_1', 'c_8', 'c_15', 'c_16', 'c_20', 'c_25'])
                
socar_strange_1hot = pd.get_dummies(socar, columns=cat_attribs)

In [48]:
socar_strange_1hot

,c_1,c_8,c_15,c_16,c_20,c_25,c_2_1,c_2_2,c_2_3,c_2_4,...,c_22_2,c_23_0,c_23_1,c_24_-1,c_24_0,c_24_1,c_24_2,c_24_3,c_24_4,c_24_5
0,0,100,1.302547e+05,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0
1,0,0,0.000000e+00,0,2,1,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,0,100,2.581306e+05,12925040,2,1,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
3,0,0,1.341009e+06,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0
4,0,100,5.240306e+05,1209410,2,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,100,0.000000e+00,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0
15996,0,100,6.184394e+05,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0
15997,0,100,0.000000e+00,0,1,1,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0
15998,0,100,0.000000e+00,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0


#### < 9. 아홉번째 방법 (NULL 유지 , SMOTE, 스케일링 없음) >

- (1) NULL 유지, (2) 데이터 분리

In [51]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar_strange_1hot[socar_strange_1hot["c_25"] == 0]
test_set = socar_strange_1hot[socar_strange_1hot["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (3) SMOTE

In [52]:
#  X : test column = 0 , y : test column = 1

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)  

In [53]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (4) 스케일링 없음

#### < 10. 열번째 방법 (NULL 유지 , SMOTE, 스케일링 (MinMax)) >

- (1) NULL 유지, (2) 데이터 분리

In [54]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar_strange_1hot[socar_strange_1hot["c_25"] == 0]
test_set = socar_strange_1hot[socar_strange_1hot["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (3) SMOTE

In [55]:
#  X : test column = 0 , y : test column = 1

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)  

In [56]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (4) 스케일링 (MinMax)

In [57]:
from sklearn.preprocessing import MinMaxScaler

MMS = MinMaxScaler()

MMS.fit(X)

X_mms = MMS.transform(X)

In [58]:
X_mms_pd10 = pd.DataFrame(X_mms, columns=X.columns)
X_mms_pd10

,c_8,c_15,c_16,c_20,c_2_1,c_2_2,c_2_3,c_2_4,c_2_5,c_3_0,...,c_22_2,c_23_0,c_23_1,c_24_-1,c_24_0,c_24_1,c_24_2,c_24_3,c_24_4,c_24_5
0,1.0,0.013409,0.000000,0.027027,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.138052,0.000000,0.027027,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.053947,0.012769,0.054054,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.000000,0.012852,0.027027,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.027027,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,1.0,0.068696,0.000000,0.027027,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12834,1.0,0.000000,0.000000,0.027027,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12835,1.0,0.063666,0.000000,0.027027,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12836,1.0,0.000000,0.000000,0.027027,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


#### < 11. 열한번째 방법 (NULL 유지 , SMOTE, 스케일링 (Standard)) >

- (1) NULL 유지, (2) 데이터 분리

In [59]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar_strange_1hot[socar_strange_1hot["c_25"] == 0]
test_set = socar_strange_1hot[socar_strange_1hot["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (3) SMOTE

In [60]:
#  X : test column = 0 , y : test column = 1

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)  

In [61]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (4) 스케일링 (Standard)

In [62]:
from sklearn.preprocessing import StandardScaler

SS = StandardScaler()

SS.fit(X)

X_ss = SS.transform(X)

In [63]:
X_ss_pd11 = pd.DataFrame(X_ss, columns=X.columns)
X_ss_pd11

,c_8,c_15,c_16,c_20,c_2_1,c_2_2,c_2_3,c_2_4,c_2_5,c_3_0,...,c_22_2,c_23_0,c_23_1,c_24_-1,c_24_0,c_24_1,c_24_2,c_24_3,c_24_4,c_24_5
0,0.368445,-0.296987,-0.273447,-0.381907,-0.846216,0.968407,-0.19857,-0.089492,-0.145737,-2.371572,...,-0.255896,0.574652,-0.574652,0.571774,-0.343937,-0.317999,-0.173700,-0.110909,-0.075622,-0.040478
1,-3.004413,2.053374,-0.273447,-0.381907,-0.846216,-1.032624,5.03601,-0.089492,-0.145737,-2.371572,...,-0.255896,0.574652,-0.574652,0.571774,-0.343937,-0.317999,-0.173700,-0.110909,-0.075622,-0.040478
2,0.368445,0.467425,0.120768,0.671804,1.181732,-1.032624,-0.19857,-0.089492,-0.145737,0.421661,...,-0.255896,0.574652,-0.574652,0.571774,-0.343937,-0.317999,-0.173700,-0.110909,-0.075622,-0.040478
3,0.368445,-0.549843,0.123334,-0.381907,1.181732,-1.032624,-0.19857,-0.089492,-0.145737,-2.371572,...,3.907842,-1.740184,1.740184,-1.748944,-0.343937,-0.317999,5.757049,-0.110909,-0.075622,-0.040478
4,-3.004413,-0.549843,-0.273447,-0.381907,-0.846216,0.968407,-0.19857,-0.089492,-0.145737,0.421661,...,-0.255896,-1.740184,1.740184,-1.748944,-0.343937,-0.317999,5.757049,-0.110909,-0.075622,-0.040478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,0.368445,0.745531,-0.273447,-0.381907,-0.846216,0.968407,-0.19857,-0.089492,-0.145737,0.421661,...,-0.255896,0.574652,-0.574652,0.571774,-0.343937,-0.317999,-0.173700,-0.110909,-0.075622,-0.040478
12834,0.368445,-0.549843,-0.273447,-0.381907,-0.846216,0.968407,-0.19857,-0.089492,-0.145737,0.421661,...,-0.255896,0.574652,-0.574652,0.571774,-0.343937,-0.317999,-0.173700,-0.110909,-0.075622,-0.040478
12835,0.368445,0.650695,-0.273447,-0.381907,-0.846216,0.968407,-0.19857,-0.089492,-0.145737,0.421661,...,-0.255896,0.574652,-0.574652,0.571774,-0.343937,-0.317999,-0.173700,-0.110909,-0.075622,-0.040478
12836,0.368445,-0.549843,-0.273447,-0.381907,-0.846216,0.968407,-0.19857,-0.089492,-0.145737,0.421661,...,-0.255896,0.574652,-0.574652,0.571774,-0.343937,-0.317999,-0.173700,-0.110909,-0.075622,-0.040478


#### < 12. 열두번째 방법 (NULL 유지 , SMOTE, 스케일링 (Robust)) >

- (1) NULL 유지, (2) 데이터 분리

In [64]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar_strange_1hot[socar_strange_1hot["c_25"] == 0]
test_set = socar_strange_1hot[socar_strange_1hot["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (3) SMOTE

In [65]:
#  X : test column = 0 , y : test column = 1

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)  

In [66]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (4) 스케일링 (Robust)

In [67]:
from sklearn.preprocessing import RobustScaler

RS = RobustScaler()

RS.fit(X)

X_rs = RS.transform(X)

In [68]:
X_ss_pd12 = pd.DataFrame(X_rs, columns=X.columns)
X_ss_pd12

,c_8,c_15,c_16,c_20,c_2_1,c_2_2,c_2_3,c_2_4,c_2_5,c_3_0,...,c_22_2,c_23_0,c_23_1,c_24_-1,c_24_0,c_24_1,c_24_2,c_24_3,c_24_4,c_24_5
0,0.0,-0.161577,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-100.0,3.179712,0.000000,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.925116,2.175491,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,-0.521038,2.189648,0.0,1.0,-1.0,0.0,0.0,0.0,-1.0,...,1.0,-1.0,1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-100.0,-0.521038,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,0.0,1.320474,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12834,0.0,-0.521038,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12835,0.0,1.185654,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12836,0.0,-0.521038,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### < 13. 열세번째 방법 (NULL 삭제 , SMOTE, 스케일링 없음) >

- null값이 들어있는 컬럼
    - c_14 : -1 - 36개
    - c_15 : 0 - 6006개
    - c_16 : 0 - 10424개
    - c_17 : 5(확인불가) - 288개
    - c_21 : 0 - 11384개
    - c_22 : 0 - 11325개
    - c_23 :  0 - 11328개
    - c_24 :  -1 - 11358개

- (1) NULL값 삭제

In [69]:
socar_strange_index_delete

,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_18,c_19,c_20,c_25
0,0,2,1,3,0,2,0,100,2,0,0,4,2,0,0,1,0
1,0,1,0,4,0,4,0,0,3,1,0,1,4,1,1,2,1
2,0,1,0,4,1,4,0,100,3,1,0,1,4,1,0,2,1
3,0,3,1,3,0,4,0,0,2,0,0,3,3,1,0,1,0
4,0,1,0,1,0,1,0,100,1,0,0,1,1,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,2,0,2,0,2,0,100,2,0,0,1,1,1,0,1,0
15996,0,2,0,2,1,4,0,100,1,1,0,5,2,1,0,1,0
15997,0,2,1,2,0,4,0,100,1,0,0,1,3,0,0,1,1
15998,0,2,0,2,0,2,0,100,2,0,0,6,1,1,0,1,0


- (2) 원핫인코딩 적용

In [70]:
cat_attribs = socar_strange_index_delete.columns.drop(['c_1', 'c_8', 'c_20', 'c_25'])

socar_strange_delete_1hot = pd.get_dummies(socar_strange_index_delete, columns=cat_attribs)

In [71]:
socar_strange_delete_1hot

,c_1,c_8,c_20,c_25,c_2_1,c_2_2,c_2_3,c_2_4,c_2_5,c_3_0,...,c_12_6,c_13_1,c_13_2,c_13_3,c_13_4,c_13_5,c_18_0,c_18_1,c_19_0,c_19_1
0,0,100,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
1,0,0,2,1,1,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,1
2,0,100,2,1,1,0,0,0,0,1,...,0,0,0,0,1,0,0,1,1,0
3,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,1,0
4,0,100,2,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,100,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,1,1,0
15996,0,100,1,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,1,1,0
15997,0,100,1,1,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
15998,0,100,1,0,0,1,0,0,0,1,...,1,1,0,0,0,0,0,1,1,0


- (3) 데이터 분리

In [72]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar_strange_delete_1hot[socar_strange_delete_1hot["c_25"] == 0]
test_set = socar_strange_delete_1hot[socar_strange_delete_1hot["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (4) SMOTE

In [73]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)

In [74]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (5) 스케일링 없음

#### < 14. 열네번쨰 방법 (NULL 삭제 , SMOTE, 스케일링 (MInMax)) >

- (1) NULL 데이터 삭제, (2) 원핫인코딩 적용,  (3) 데이터 분리

In [75]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar_strange_delete_1hot[socar_strange_delete_1hot["c_25"] == 0]
test_set = socar_strange_delete_1hot[socar_strange_delete_1hot["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (4) SMOTE

In [76]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)

In [77]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (5) 스케일링

In [78]:
from sklearn.preprocessing import MinMaxScaler

MMS = MinMaxScaler()

MMS.fit(X)

X_mms = MMS.transform(X)

In [79]:
X_mms_pd14 = pd.DataFrame(X_mms, columns=X.columns)
X_mms_pd14

,c_8,c_20,c_2_1,c_2_2,c_2_3,c_2_4,c_2_5,c_3_0,c_3_1,c_4_1,...,c_12_6,c_13_1,c_13_2,c_13_3,c_13_4,c_13_5,c_18_0,c_18_1,c_19_0,c_19_1
0,1.0,0.027027,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.027027,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,1.0,0.054054,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,1.0,0.027027,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.027027,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,1.0,0.027027,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
12834,1.0,0.027027,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
12835,1.0,0.027027,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
12836,1.0,0.027027,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


#### < 15. 열다섯번째 방법 (NULL 삭제 , SMOTE, 스케일링 (Standard)) >

- (1) NULL 데이터 삭제, (2) 원핫인코딩 적용,  (3) 데이터 분리

In [81]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar_strange_delete_1hot[socar_strange_delete_1hot["c_25"] == 0]
test_set = socar_strange_delete_1hot[socar_strange_delete_1hot["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (4) SMOTE

In [82]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)

In [83]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (5) 스케일링

In [84]:
from sklearn.preprocessing import StandardScaler

SS = StandardScaler()

SS.fit(X)

X_ss = SS.transform(X)

In [85]:
X_ss_pd15 = pd.DataFrame(X_ss, columns=X.columns)
X_ss_pd15

,c_8,c_20,c_2_1,c_2_2,c_2_3,c_2_4,c_2_5,c_3_0,c_3_1,c_4_1,...,c_12_6,c_13_1,c_13_2,c_13_3,c_13_4,c_13_5,c_18_0,c_18_1,c_19_0,c_19_1
0,0.368445,-0.381907,-0.846216,0.968407,-0.19857,-0.089492,-0.145737,-2.371572,2.371572,-0.532247,...,-0.468788,-0.754230,2.008118,-0.655334,-0.334371,-0.197933,0.696804,-0.696804,0.45426,-0.45426
1,-3.004413,-0.381907,-0.846216,-1.032624,5.03601,-0.089492,-0.145737,-2.371572,2.371572,-0.532247,...,-0.468788,-0.754230,-0.497979,1.525940,-0.334371,-0.197933,-1.435124,1.435124,0.45426,-0.45426
2,0.368445,0.671804,1.181732,-1.032624,-0.19857,-0.089492,-0.145737,0.421661,-0.421661,1.878828,...,-0.468788,1.325856,-0.497979,-0.655334,-0.334371,-0.197933,0.696804,-0.696804,0.45426,-0.45426
3,0.368445,-0.381907,1.181732,-1.032624,-0.19857,-0.089492,-0.145737,-2.371572,2.371572,-0.532247,...,2.133159,-0.754230,-0.497979,1.525940,-0.334371,-0.197933,0.696804,-0.696804,0.45426,-0.45426
4,-3.004413,-0.381907,-0.846216,0.968407,-0.19857,-0.089492,-0.145737,0.421661,-0.421661,-0.532247,...,-0.468788,-0.754230,2.008118,-0.655334,-0.334371,-0.197933,0.696804,-0.696804,0.45426,-0.45426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,0.368445,-0.381907,-0.846216,0.968407,-0.19857,-0.089492,-0.145737,0.421661,-0.421661,-0.532247,...,-0.468788,1.325856,-0.497979,-0.655334,-0.334371,-0.197933,0.696804,-0.696804,0.45426,-0.45426
12834,0.368445,-0.381907,-0.846216,0.968407,-0.19857,-0.089492,-0.145737,0.421661,-0.421661,-0.532247,...,-0.468788,1.325856,-0.497979,-0.655334,-0.334371,-0.197933,-1.435124,1.435124,0.45426,-0.45426
12835,0.368445,-0.381907,-0.846216,0.968407,-0.19857,-0.089492,-0.145737,0.421661,-0.421661,-0.532247,...,-0.468788,-0.754230,2.008118,-0.655334,-0.334371,-0.197933,-1.435124,1.435124,0.45426,-0.45426
12836,0.368445,-0.381907,-0.846216,0.968407,-0.19857,-0.089492,-0.145737,0.421661,-0.421661,-0.532247,...,2.133159,1.325856,-0.497979,-0.655334,-0.334371,-0.197933,-1.435124,1.435124,0.45426,-0.45426


#### < 16. 열여섯번째 방법 (NULL 삭제 , SMOTE, 스케일링 (Robust)) >

- (1) NULL 데이터 삭제, (2) 원핫인코딩 적용,  (3) 데이터 분리

In [86]:
from sklearn.model_selection import train_test_split

# train_set, test_set 분리
train_set = socar_strange_delete_1hot[socar_strange_delete_1hot["c_25"] == 0]
test_set = socar_strange_delete_1hot[socar_strange_delete_1hot["c_25"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("c_25", axis=1)  # test_set = 0인 애들이고, test_set 컬럼을 삭제
test_set = test_set.drop("c_25", axis=1)    # test_set = 1인 애들이고, test_set 컬럼을 삭제

# features와 target분리
X = train_set.drop("c_1", axis=1)         # train_set에서 y라벨 데이터 지움
y = train_set["c_1"]                      # 라벨데이터만 가지고 y변수에 할당

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)
X_test, y_test = test_set.drop("c_1", axis=1), test_set["c_1"]

- (4) SMOTE

In [87]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_resample(X, y)

In [88]:
print(np.unique(y, return_counts=True))
print(np.unique(y_train_over, return_counts=True))

(array([0, 1], dtype=int64), array([12804,    34], dtype=int64))
(array([0, 1], dtype=int64), array([12804, 12804], dtype=int64))


- (5) 스케일링

In [89]:
from sklearn.preprocessing import RobustScaler

RS = RobustScaler()

RS.fit(X)

X_rs = RS.transform(X)

In [90]:
X_rs_pd16 = pd.DataFrame(X_rs, columns=X.columns)
X_rs_pd16

,c_8,c_20,c_2_1,c_2_2,c_2_3,c_2_4,c_2_5,c_3_0,c_3_1,c_4_1,...,c_12_6,c_13_1,c_13_2,c_13_3,c_13_4,c_13_5,c_18_0,c_18_1,c_19_0,c_19_1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-100.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,1.0,0.0,0.0
2,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,-1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0
12835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0
12836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0


#### -여기까지 하면 (이상 index 삭제) x (원핫인코딩(유,무)) x (NULL데이터 (유,무)) x ((스케일링 4가지)) => 16가지 끝